# Data cleaning for SEER 5y incidence

In [26]:
import numpy as np
import pandas as pd
import os
os.getcwd() 

In [29]:
seer_inc = pd.read_csv("../data/pdac_seer17_incidence_5y.csv")
seer_inc.iloc[:5]

,Year Dx (2000-2021 group only) for DevCan,Age recode with <1 year olds DevCan,Combined Summary Stage (2004+),Crude Rate,Count,Population
0,2000-2021,00 years,In situ,0.0,0,23639727
1,2000-2021,00 years,Localized,0.0,0,23639727
2,2000-2021,00 years,Regional,0.0,0,23639727
3,2000-2021,00 years,Distant,0.0,0,23639727
4,2000-2021,00 years,NaN,0.0,0,23639727


In [31]:
# Pivot the data
pivot_df = seer_inc.pivot(
    index='Age recode with <1 year olds DevCan', 
    columns='Combined Summary Stage (2004+)', 
    values='Crude Rate'
)

pivot_df.columns = [f"{col} Rate" for col in pivot_df.columns]
pivot_df.reset_index(inplace=True)
pivot_df['Age'] = pivot_df['Age recode with <1 year olds DevCan'].str[:2].astype(int)


In [32]:
pivot_df.drop(['nan Rate', 'In situ Rate', 'Blank(s) Rate', 'Age recode with <1 year olds DevCan'], axis=1, inplace=True)

Redistribute unstaged cancer incidence over local, regional, and distant

In [33]:
pivot_df['LRD Rate'] = pivot_df['Localized Rate'] + pivot_df['Regional Rate'] + pivot_df['Distant Rate']
pivot_df['LRDU Rate'] = pivot_df['Localized Rate'] + pivot_df['Regional Rate'] + pivot_df['Distant Rate'] + pivot_df['Unknown/unstaged Rate']
pivot_df['pLocal'] = pivot_df['Localized Rate'] / pivot_df['LRD Rate']
pivot_df['pRegional'] = pivot_df['Regional Rate'] / pivot_df['LRD Rate']
pivot_df['pDistant'] = pivot_df['Distant Rate'] / pivot_df['LRD Rate']
pivot_df['LocalU'] = pivot_df['Localized Rate'] + pivot_df['Unknown/unstaged Rate'] * pivot_df['pLocal']
pivot_df['RegionalU'] = pivot_df['Regional Rate'] + pivot_df['Unknown/unstaged Rate'] * pivot_df['pRegional']
pivot_df['DistantU'] = pivot_df['Distant Rate'] + pivot_df['Unknown/unstaged Rate'] * pivot_df['pDistant']

In [37]:
pivot_df = pivot_df[pivot_df['Age']>=20]
pivot_df = pivot_df[pivot_df['Age']<85]

In [38]:
pivot_df.to_csv("../data/seer_incidence_1y.csv")